In [ ]:
from __future__ import print_function
import os.path
import pandas as pd
import sys
sys.path.insert(0, '../../')
%load_ext autoreload
from JKBio.Helper import *
%autoreload 2
%load_ext rpy2.ipython
import numpy as np
from bokeh.plotting import *
from bokeh.models import HoverTool
output_notebook()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from JKBio.helper import pyDESeq2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from umap import UMAP

downloading their data

# we first need to recreate out RNA matrix from slamdunk's tcount files

In [ ]:
ls ../../data/slam/

In [ ]:
location= '../../data/slam/IRF2BP2degraded_v1/count/'

In [ ]:
files = ! ls $location*.tsv
files

In [ ]:
prev = -2
flag_var= 100
data={}
for file in files:
    data[file.split('.')[-3].split('/')[-1]]= pd.read_csv(file, sep='\t', comment='#',header=0)
for k,val in data.items():
    if len(set(val.Name)) != prev and prev!=-2: 
        raise ValueError('we do not have the same number of genes in each file')
    prev = len(set(val.Name))
readcounts = {i:[0]*len(data) for i in val.Name.unique()}
tccounts = {i:[0]*len(data) for i in val.Name.unique()}
data.keys()

In [ ]:
for n, (_,val) in enumerate(data.items()):
    val = val.sort_values(by="Name")
    j  = 0
    print('              ',end='\r')
    readcount = [val.iloc[0].ReadCount]
    tccount = [val.iloc[0].TcReadCount]
    prevname = val.iloc[0].Name
    for _, v in val.iloc[1:].iterrows():
        if v.Name==4609:
            print(v.ReadCount,v.TcReadCount)
            print(readcount,tccount)
        if v.Name == prevname:
            readcount.append(v.ReadCount)
            tccount.append(v.TcReadCount)
        else:
            readcounts[prevname][n] = np.sum(readcount)
            tccounts[prevname][n] = np.sum(tccount)
            #if np.var(readcount) > flag_var:
            #    print("pb with "+str(v.Name))
            prevname=v.Name
            j+=1
            print(j,end='\r')
            readcount = [v.ReadCount]
            tccount = [v.TcReadCount]
            
files = [*data]
readcounts = pd.DataFrame(data=readcounts,columns=val.Name.unique(),index=data.keys()).T
tccounts = pd.DataFrame(data=tccounts,columns=val.Name.unique(),index=data.keys()).T
names, _ = convertGenes(readcounts.index.tolist(), from_idtype="entrez_id", to_idtype="symbol")
readcounts.index = names
names, _ = convertGenes(tccounts.index.tolist(), from_idtype="entrez_id", to_idtype="symbol")
tccounts.index = names

# or get it directly if available

In [ ]:
readcounts = pd.read_csv('../../data/slam/readcounts_prev.csv',index_col='genes')
tccounts = pd.read_csv('../../data/slam/tccounts_prev.csv',index_col='genes')

# get only the most variable genes

In [ ]:
readcounts.values.mean()

In [ ]:
len(np.argwhere(tccounts.values.var(1)>=800).ravel())

In [ ]:
todrop = np.argwhere(tccounts.values.var(1)>=800).ravel()
tccounts = tccounts.iloc[todrop]
readcounts = readcounts.iloc[todrop]

In [ ]:
#todrop=np.argwhere(readcounts.values.max(1)>=10).ravel()
#tccounts = tccounts.iloc[todrop]
#readcounts = readcounts.iloc[todrop]

In [ ]:
len(readcounts)

## SLAM-seq
All SLAM-seq assays were performed at 60-70% confluency for adherent cells or
60% of the maximum cell density counted on a hemocytometer for suspension cells. 5-7h
prior to each assay, growth medium was aspirated and replaced. Unless stated otherwise,
cells were pre-treated with indicated small molecule inhibitors or 100µM IAA for 30 min
to pre-establish full target inhibition or degradation. Newly synthesized RNA was labeled
for indicated time spans (45 min or 60 min) at a final concentration of 100µM 4-
thiouridine (4sU, Carbosynth). Adherent cells were harvested by direct snap-freezing of
plates on dry ice. Suspension cells were spun down and immediately snap-frozen. RNA
extraction was performed using the RNeasy Plus Mini Kit (Qiagen). 

Total RNA was subjected to alkylation by iodoacetamide (Sigma, 10mM) for 15 min and RNA was repurified by ethanol precipitation. 500ng alkylated RNA were used as input for generating
3’-end mRNA sequencing libraries using a commercially available kit (QuantSeq 3′
mRNA-Seq Library Prep Kit FWD for Illumina and PCR Add-on Kit for Illumina,
Lexogen). Deep sequencing was performed using HiSeq1500 and HiSeq2500 platforms
(Illumina).

0.06724463
0.04916348
0.03728357
0.05075007
0.03331248
0.0346662
0.03265504 
0.04416262
0.05574821
0.05845953
0.05531433
0.06175393
0.03181749 
0.02940226 
0.0306263
0.04004278

In [ ]:
tccounts

In [ ]:
tccounts.mean()

# lets look at the similarity accross replicates

we have some outliers, else it seems to make some sense and on average, to be extremelly similar!

In [ ]:
readcounts.loc['MYC']

In [ ]:
%matplotlib inline
sns.pairplot(tccounts)

In [ ]:
%matplotlib inline
sns.heatmap(tccounts.corr(), 
            xticklabels=tccounts.columns,
            yticklabels=tccounts.columns)

In [ ]:
scatter(PCA(2).fit_transform(tccounts.values.T),labels=tccounts.columns, radi=700)

very strangely, we have in non euclidean space a structure that matches the labels we have. 
We hypotesized that noise might explain the difference in correlation that we see, however it would not explain that in euclidean space, things cluster still very well but differently. 

As a matter of fact. real noise would not change the correlation at all. This noise has a structure. One thing can explain read count that, at low frequency transcripts, especially create a specific pattern: contamination

In [ ]:
todrop = ['mr78.MV411_IRF2BP2.DMSO_6h.r1', 'mr76.MV411_IRF2BP2.DMSO_2h.r3', 'mr80.MV411_IRF2BP2.DMSO_6h.r3', 'mr74.MV411_IRF2BP2.DMSO_2h.r1', 'mr84.MV411_IRF2BP2.VHL_2h.r3']

In [ ]:
tccounts.columns = [i.replace('-','.') for i in tccounts.columns]
readcounts.columns = [i.replace('-','.') for i in readcounts.columns]

In [ ]:
tccounts = tccounts.drop(columns=[c for c in tccounts.columns if c in todrop])
readcounts = readcounts.drop(columns=[c for c in readcounts.columns if c in todrop])

In [ ]:
readcounts

In [ ]:
%matplotlib inline
sns.pairplot(tccounts)

In [ ]:
%matplotlib inline
sns.heatmap(tccounts.corr(), 
            xticklabels=tccounts.columns,
            yticklabels=tccounts.columns)

In [ ]:
%matplotlib inline
sort = [4,9,7,0,6,5,2,3,8,1,10]
sns.heatmap(tccounts[tccounts.columns[sort]].corr(), 
            xticklabels=tccounts.columns[sort].values,
            yticklabels=tccounts.columns[sort].values)

In [ ]:
%matplotlib inline
sns.heatmap(readcounts.corr(), 
            xticklabels=readcounts.columns,
            yticklabels=readcounts.columns)

In [ ]:
%matplotlib inline
sort = [4,9,7,0,6,5,2,3,8,1,10]
sns.heatmap(readcounts[readcounts.columns[sort]].corr(), 
            xticklabels=readcounts.columns.values[sort],
            yticklabels=readcounts.columns.values[sort])

read the paper for MV411 QC steps.

### A quick detour to look for clustering

## Getting the Core TF information

In [ ]:
ctf=pd.read_csv('../data/CTF.csv',header=None)[0].values.tolist()
ctf

In [ ]:
set(ctf) & set(tccounts.index)

In [ ]:
ctfpos = [val for val in tccounts.index if val in ctf]
notctfpos = [val for val in tccounts.index if val not in ctf]

In [ ]:
ctfpos.extend(['EGR1','SERTAD1'])

We find a CTF not in the dataset

# Let's look at how they change accross time

how core transcription factors change accross time when the cell is with JQ1 conditions

we are looking at the difference between production of each CTF compared to the average RNA production with JQ1 and without JQ1

## DESEQ ANALYSIS

In [ ]:
readcounts["genes"] = readcounts.index
readcounts = readcounts.reset_index(drop=True)
tccounts["genes"] = tccounts.index
tccounts = tccounts.reset_index(drop=True)

In [ ]:
tccounts.iloc[140]

In [ ]:
tccounts.columns

In [ ]:
design_2 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','VHL'], 
                      data=np.array([[1,1,0,0,0,0,0,0,0,0,0],
                                     [0,0,0,0,1,1,1,0,0,0,0]],dtype=bool).T)

In [ ]:
design_6 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','VHL'], 
                      data=np.array([[0,0,1,1,0,0,0,0,0,0,0],
                                     [0,0,0,0,0,0,0,1,1,1,1]],dtype=bool).T)

### Differential gene expression analysis, PCA and GO-term enrichment

For gene-level analysis, raw reads mapped to different UTR annotations of the same gene were summed up by Entrez Gene ID. Pilot studies of K562 cells with kinase inhibitors were performed as single experiments. 

Analysis of differential gene expression was restricted to genes with ≥ 10 reads in at least one condition for 50bp sequencing runs (flavopiridol and DMSO) or ≥ 20 reads in at least one condition for 100bp sequencing runs (mk2206, trametinib, nilotinib, trametinib + mk2206 and DMSO). For estimating differential expression, a pseudo-count of 1 raw read was added to all genes.

#### All other SLAM-seq experiments were performed in triplicates and analyzed as follows. 
Differential gene expression calling was performed on raw read counts with ≥ 2 T>C conversions using DESeq2 (version 1.14.1) with default settings, and with size factors estimated on corresponding total mRNA reads for global normalization.

Downstream analysis was restricted to genes passing all internal filters for FDR estimation by DESeq2. Principal component analysis was performed after variance
stabilizing transformation on the 500 most variable genes across all conditions of a given
experiment. GO-term enrichment analysis was performed on genes significantly and strongly downregulated (FDR ≤ 0.1, log2FC ≤ -1) in SLAM-seq upon IAA-treatment in K562MYC-AID + Tir1 by the PANTHER Overrepresentation Test (Fisher's Exact with FDR multiple test correction, release 20171205, http://pantherdb.org) on GO Ontology database Released 2017-12-27.

In [ ]:
tccounts[tccounts.genes=='MYC']

In [ ]:
deseq_2 = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([1,1,0,0,1,1,1,0,0,0,0,1],np.bool)]],
                         design_matrix=design_2[np.array([1,1,0,0,1,1,1,0,0,0,0],np.bool)],
                         design_formula="~DMSO - VHL",
                         gene_column="genes")

In [ ]:
deseq_6 = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([0,0,1,1,0,0,0,1,1,1,1,1],np.bool)]],
                         design_matrix=design_6[np.array([0,0,1,1,0,0,0,1,1,1,1],np.bool)],
                         design_formula="~DMSO - VHL",
                         gene_column="genes")

why is DESeq still not finding under expression even when we clearly see underexpression at the total read count

In [ ]:
readcounts.values[:,0:2].mean(1).sum() / readcounts.values[:,4:7].mean(1).sum()

In [ ]:
scaling2 = np.log2(readcounts.values[:,0:2].mean(1).sum() / readcounts.values[:,4:7].mean(1).sum())
scaling2

In [ ]:
scaling6 = np.log2(readcounts.values[:,2:4].mean(1).sum() / readcounts.values[:,7:11].mean(1).sum())
scaling6

First we can clearly see that we have a big issue here in the number of reads that were sequenced. we have much more reads in 

In [ ]:
deseq_2.run_estimate_size_factors(normMatrix = 3/np.array([np.exp(np.log(
    readcounts[readcounts.columns[
        np.array([1,1,0,0,0,0,0,0,0,0,0,0],np.bool)]].values+1).mean(1)),np.exp(np.log(
    readcounts[readcounts.columns[
        np.array([0,0,0,0,1,1,1,0,0,0,0,0],np.bool)]].values+1).mean(1))]))

In [ ]:
deseq_6.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[
        np.array([0,0,1,1,0,0,0,0,0,0,0,0],np.bool)]].values+1),1)))
deseq_2.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[
        np.array([1,1,0,0,0,0,0,0,0,0,0,0],np.bool)]].values+1),1)))

In [ ]:
val2 = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[
        np.array([1,1,0,0,0,0,0,0,0,0,0,0],np.bool)]].values+1),1))
val2 = val2 - (scaling2*val2)
deseq_2.run_estimate_size_factors(geoMeans = val2)
val6 = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[
        np.array([0,0,1,1,0,0,0,0,0,0,0,0],np.bool)]].values+1),1))
val6 = val6 - (scaling6*val6)
deseq_6.run_estimate_size_factors(geoMeans = val6)

In [ ]:
deseq_2.run_deseq()
deseq_6.run_deseq()

In [ ]:
deseq_2.get_deseq_result()
deseq_6.get_deseq_result()

In [ ]:
res2 = deseq_2.deseq_result
res6 = deseq_6.deseq_result

In [ ]:
res2.pvalue = np.nan_to_num(np.array(res2.pvalue), 1)
res2.log2FoldChange = np.nan_to_num(np.array(res2.log2FoldChange), 0)
res6.pvalue = np.nan_to_num(np.array(res6.pvalue), 1)
res6.log2FoldChange = np.nan_to_num(np.array(res6.log2FoldChange), 0)

In [ ]:
res2.log2FoldChange = -res2.log2FoldChange
res6.log2FoldChange = -res6.log2FoldChange

In [ ]:
res2["gene_id"] = res2.genes
res6["gene_id"] = res6.genes

In [ ]:
tccounts[tccounts.genes=="THBD"]

In [ ]:
res6.log2FoldChange += scaling6
res2.log2FoldChange += scaling2

In [ ]:
show(volcano(res6,tohighlight=ctf, searchbox=True, maxvalue=20,minpval=1, minlogfold=0.1))

In [ ]:
show(volcano(res2,tohighlight=ctf, searchbox=True, minpval=0.5, minlogfold=0.05))

we can conclude that we get similar results to the slamseq myc paper although it seems that our values are a bit skewed toward higher expression than what is on the slamseq paper. It mightt be explained by the pseudo count of 1 that I did not set. Because I think it would highly bias the DESeq algorithm.

In [ ]:
interferon_sig =  ["AAAS", "ABCE1", "AC004551.1", "ADAR", "ARIH1", "B2M", "BST2", "CAMK2A", "CAMK2B", "CAMK2D", "CAMK2G", "CD44", "CIITA", "DDX58", "EGR1", "EIF2AK2", "EIF4A1", "EIF4A2", "EIF4A3", "EIF4E", "EIF4E2", "EIF4E3", "EIF4G1", "EIF4G2", "EIF4G3", "FCGR1A", "FCGR1B", "FLNA", "FLNB", "GBP1", "GBP2", "GBP3", "GBP4", "GBP5", "GBP6", "GBP7", "HERC5", "HLA-A", "HLA-B", "HLA-C", "HLA-DPA1", "HLA-DPB1", "HLA-DQA1", "HLA-DQA2", "HLA-DQB1", "HLA-DQB2", "HLA-DRA", "HLA-DRB1", "HLA-DRB3", "HLA-DRB4", "HLA-DRB5", "HLA-E", "HLA-F", "HLA-G", "HLA-H", "ICAM1", "IFI27", "IFI30", "IFI35", "IFI6", "IFIT1", "IFIT2", "IFIT3", "IFITM1", "IFITM2", "IFITM3", "IFNA1", "IFNA10", "IFNA13", "IFNA14", "IFNA16", "IFNA17", "IFNA2", "IFNA21", "IFNA4", "IFNA5", "IFNA6", "IFNA7", "IFNA8", "IFNAR1", "IFNAR2", "IFNB1", "IFNG", "IFNGR1", "IFNGR2", "IP6K2", "IRF1", "IRF2", "IRF3", "IRF4", "IRF5", "IRF6", "IRF7", "IRF8", "IRF9", "ISG15", "ISG20", "JAK1", "JAK2", "KPNA1", "KPNA2", "KPNA3", "KPNA4", "KPNA5", "KPNA7", "KPNB1", "MAPK3", "MID1", "MT2A", "MX1", "MX2", "NCAM1", "NDC1", "NEDD4", "NUP107", "NUP133", "NUP153", "NUP155", "NUP160", "NUP188", "NUP205", "NUP210", "NUP214", "NUP35", "NUP37", "NUP43", "NUP50", "NUP54", "NUP58", "NUP62", "NUP85", "NUP88", "NUP93", "NUP98", "NUPL2", "OAS2", "OAS3", "OASL", "PDE12", "PIAS1", "PIN1", "PLCG1", "PML", "POM121", "POM121C", "PPM1B", "PRKCD", "PSMB8", "PTAFR", "PTPN1", "PTPN11", "PTPN2", "PTPN6", "RAE1", "RANBP2", "RNASEL", "RPS27A", "RSAD2", "SAMHD1", "SEC13", "SEH1L", "SOCS1", "SOCS3", "SP100", "STAT1", "STAT2", "SUMO1", "TPR", "TRIM10", "TRIM14", "TRIM17", "TRIM2", "TRIM21", "TRIM22", "TRIM25", "TRIM26", "TRIM29", "TRIM3", "TRIM31", "TRIM34", "TRIM35", "TRIM38", "TRIM45", "TRIM46", "TRIM48", "TRIM5", "TRIM6", "TRIM62", "TRIM68", "TRIM8", "TYK2", "UBA52", "UBA7", "UBB", "UBC", "UBE2E1", "UBE2L6", "UBE2N", "USP18", "USP41", "VCAM1", "XAF1"]

In [ ]:
show(volcano(res6,tohighlight=interferon_sig, searchbox=True, maxvalue=20,minpval=1, minlogfold=0.1))

In [ ]:
tccounts[]

In [ ]:
tccounts[readcounts.columns[np.array([0,0,1,1,0,0,0,1,1,1,1,0],np.bool)]]

In [ ]:
! ls ../data/genesets/apoptosis.gmt

In [ ]:
tccounts = tccounts.set_index('genes')

In [ ]:
gp.get_library_name()

In [ ]:
import gseapy as gp
from gseapy.plot import gseaplot, heatmap, barplot, dotplot
#cls = ['DMSO','DMSO','Condition','Condition','Condition','Condition']
cls = [0,0,1,1,1,1]
res = gp.gsea(data=tccounts[tccounts.columns[np.array([0,0,1,1,0,0,0,1,1,1,1],np.bool)]],
         gene_sets='../data/genesets/interferon_sig.gmt',
         cls=cls,
         no_plot=False,
         processes=6)

In [ ]:
res.res2d['Term'] = res.res2d.index

In [ ]:
sns.barplot(data=res.res2d.iloc[:25], x="es", y="Term",
                hue_order="geneset_size")

In [ ]:
gseaplot(res.ranking, term='REACTOME_INTERFERON_SIGNALING', **res.results["REACTOME_INTERFERON_SIGNALING"])